In [1]:
#Paquetes necesarios
import cv2
import numpy as np
from deepface import DeepFace
from PIL import Image

In [2]:
# Función para superponer imágenes con canal alfa
def overlay_image_alpha(img, overlay, position, alpha_mask):
    x, y = position
    h, w = overlay.shape[:2]

    # Recorte de las coordenadas para asegurar que no se salga del frame
    if x < 0:
        overlay = overlay[:, -x:]
        alpha_mask = alpha_mask[:, -x:]
        w += x
        x = 0
    if y < 0:
        overlay = overlay[-y:, :]
        alpha_mask = alpha_mask[-y:, :]
        h += y
        y = 0
    if x + w > img.shape[1]:
        overlay = overlay[:, :img.shape[1] - x]
        alpha_mask = alpha_mask[:, :img.shape[1] - x]
        w = img.shape[1] - x
    if y + h > img.shape[0]:
        overlay = overlay[:img.shape[0] - y, :]
        alpha_mask = alpha_mask[:img.shape[0] - y, :]
        h = img.shape[0] - y

    # Crear un fondo y sobreponer con la máscara alfa
    img_crop = img[y:y+h, x:x+w]
    img_crop[:] = img_crop * (1 - alpha_mask[:, :, np.newaxis] / 255.0) + overlay * (alpha_mask[:, :, np.newaxis] / 255.0)

    return img

# Función para cargar fotogramas de un GIF
def load_gif_frames(gif_path):
    pil_gif = Image.open(gif_path)
    frames = []
    try:
        while True:
            frame = pil_gif.convert("RGBA")
            frame = cv2.cvtColor(np.array(frame), cv2.COLOR_RGBA2BGRA)
            frames.append(frame)
            pil_gif.seek(pil_gif.tell() + 1)
    except EOFError:
        pass
    return frames


In [3]:
scouter = cv2.imread('scouter.png', cv2.IMREAD_UNCHANGED)
aura_index = 0

hair = cv2.imread('goku-hair.png', cv2.IMREAD_UNCHANGED)
aura_frames = load_gif_frames('aura.gif')
aura_frame_count = len(aura_frames)
frame_count = 0
changed = 0

# Captura de video en tiempo real
cap = cv2.VideoCapture(0)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height =int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fps = int(cap.get(cv2.CAP_PROP_FPS))

out = cv2.VideoWriter("kakaroto_video.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # Detección de rostros
        faces = DeepFace.extract_faces(frame, detector_backend='mtcnn')

        for face in faces:
            x, y, w, h = face['facial_area']['x'], face['facial_area']['y'], face['facial_area']['w'], face['facial_area']['h']
            face_width = w
            face_height = h

            # Análisis de emoción
            face_img = frame[y:y+h, x:x+w]
            analysis = DeepFace.analyze(face_img, actions=['emotion'], enforce_detection=False)
            emotion = analysis[0]['dominant_emotion']

            # Activar efectos si la emoción es enfado
            
            scouter_resized = cv2.resize(scouter, (int(face_width * 0.5), int(face_height * 0.3)))
            scouter_alpha = scouter_resized[:, :, 3]

            
            eye_x = face['facial_area']['right_eye'][0] - w // 3
            eye_y = face['facial_area']['right_eye'][1] - scouter_resized.shape[1] // 3

            if emotion == 'angry':
                hair_resized = cv2.resize(hair, (face_width * 3, face_height * 3))
                hair_alpha = hair_resized[:, :, 3]
                hair_x = x + face_width // 2 - hair_resized.shape[1] // 2
                hair_y = y - int(hair_resized.shape[0] * 0.7)

                aura_frame = aura_frames[aura_index]
                aura_resized = cv2.resize(aura_frame, (frame.shape[0] * 2, frame.shape[1] * 2))
                aura_alpha = aura_resized[:, :, 3]

                aura_x = x + face_width // 2 - aura_resized.shape[1] // 2
                aura_y = round(y - frame.shape[1] * 1.05)

                if (frame_count < 10):
                    if (changed == 0):
                        aura_frames = load_gif_frames('aura.gif')
                        aura_frame_count = len(aura_frames)
                        changed += 1
                    hair = cv2.imread('saiyan-hair.png', cv2.IMREAD_UNCHANGED)
                elif (frame_count < 20):
                    if (changed == 1):
                        aura_frames = load_gif_frames('aura-god.gif')
                        aura_frame_count = len(aura_frames)
                        changed += 1
                    hair = cv2.imread('god-hair.png', cv2.IMREAD_UNCHANGED)
                elif (frame_count < 30):
                    if (changed == 2):
                        aura_frames = load_gif_frames('aura-blue.gif')
                        aura_frame_count = len(aura_frames)
                        changed += 1
                    hair = cv2.imread('blue-hair.png', cv2.IMREAD_UNCHANGED)
                else:
                    if (changed == 3):
                        aura_frames = load_gif_frames('aura-instinct.gif')
                        aura_frame_count = len(aura_frames)
                        changed += 1
                    hair = cv2.imread('instinct-hair.png', cv2.IMREAD_UNCHANGED)

                # Superposición de efectos
                frame = overlay_image_alpha(frame, aura_resized[:, :, :3], (aura_x, aura_y), aura_alpha)
                frame = overlay_image_alpha(frame, scouter_resized[:, :, :3], (eye_x, eye_y), scouter_alpha)
                frame = overlay_image_alpha(frame, hair_resized[:, :, :3], (hair_x, hair_y), hair_alpha)

                frame_count += 1

                # Avanzar el índice de fotograma para animación
                aura_index = (aura_index + 1) % aura_frame_count
            else:
                hair = cv2.imread('goku-hair.png', cv2.IMREAD_UNCHANGED)
                hair_resized = cv2.resize(hair, (round(face_width * 2.6), face_height * 2))
                hair_alpha = hair_resized[:, :, 3]
                hair_x = x + face_width // 2 - hair_resized.shape[1] // 2
                hair_y = y - int(hair_resized.shape[0] * 0.65)
                frame = overlay_image_alpha(frame, scouter_resized[:, :, :3], (eye_x, eye_y), scouter_alpha)
                frame = overlay_image_alpha(frame, hair_resized[:, :, :3], (hair_x, hair_y), hair_alpha)

                

    except Exception as e:
        print("Error en detección o análisis de emociones:", e)

    out.write(frame)   

    cv2.imshow("Filtro Saiyan", frame)

    # Cambiar de detector o salir
    key = cv2.waitKey(1) & 0xFF
    if key == 27:  # Salir con ESC
        break

cap.release()
out.release()
cv2.destroyAllWindows()

Error en detección o análisis de emociones: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error en detección o análisis de emociones: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error en detección o análisis de emociones: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error en detección o análisis de emociones: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error en detección o análisis de emociones: Face could not be detected in numpy array.Please confirm that the picture is a face photo or consider to set enforce_detection param to False.
Error en detección o análisis de emociones: Face could not be det

KeyboardInterrupt: 